# Gallery Example: Erl/Erl/1 Queue

This example demonstrates an Erlang/Erlang/1 queueing system:
- **Arrivals**: Erlang distribution
- **Service**: Erlang distribution
- **Servers**: 1 server
- **Capacity**: Infinite
- **Scheduling**: FCFS

Both arrival and service processes use Erlang distributions, providing lower variability than exponential. This model also demonstrates class switching.

In [ ]:
from line_solver import *
GlobalConstants.set_verbose(VerboseLevel.STD)

In [ ]:
def gallery_erlerl1(n=5):
    """Create Erl/Erl/1 queueing model with class switching"""
    model = Network('Erl/Erl/1')
    
    # Block 1: nodes
    source = Source(model, 'Source')
    queue = Queue(model, 'Queue', SchedStrategy.FCFS)
    sink = Sink(model, 'Sink')
    
    # Block 2: classes
    oclass1 = OpenClass(model, 'Class1')
    oclass2 = OpenClass(model, 'Class2')
    
    # Erlang arrival for Class1, Class2 is disabled at source
    source.set_arrival(oclass1, Erlang.fit_mean_and_order(1, n))
    source.set_arrival(oclass2, Disabled())
    
    # Erlang service for Class1, Exponential for Class2
    queue.set_service(oclass1, Erlang.fit_mean_and_order(0.5, n))
    queue.set_service(oclass2, Exp(3))
    
    # Block 3: topology with class switching
    P = model.init_routing_matrix()
    P.add_route(oclass1, source, queue, 1.0)
    P.add_route(oclass2, queue, sink, 1.0)
    model.link(P)
    
    return model

# Create the model
model = gallery_erlerl1()

## About Erlang Distributions

The Erlang distribution is a sum of n independent exponential random variables:
- **Mean**: k/λ where k is the order
- **Variance**: k/λ²
- **SCV (squared coefficient of variation)**: 1/k

With order n=5, the SCV is 0.2, providing much less variability than exponential (SCV=1).

## About Class Switching

This model demonstrates class switching:
- Jobs arrive as Class1 (Erlang arrivals)
- After service, jobs become Class2 and leave
- Class2 cannot arrive from the source (Disabled)

In [ ]:
# Solve with multiple solvers
print("\n=== Solver Results ===")

# CTMC Solver
solver_ctmc = CTMC(model, cutoff=15)
avg_table_ctmc = solver_ctmc.avg_table()
print("\nCTMC Solver:")
print(avg_table_ctmc)

# Fluid Solver
solver_fluid = FLD(model)
avg_table_fluid = solver_fluid.avg_table()
print("\nFluid Solver:")
print(avg_table_fluid)